In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from module.prepare_data import *

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
END_TRAIN = 1913

In [4]:
if not os.path.exists(TEMP_FEATURE_PKL):
    train_df = pd.read_csv(f'{ORI_CSV_PATH}/sales_train_evaluation.csv')
#     train_df = train_df[train_df['id'].isin(IDS[:100])]
    prices_df = pd.read_csv(f'{ORI_CSV_PATH}/sell_prices.csv')
    calendar_df = pd.read_csv(f'{ORI_CSV_PATH}/calendar.csv')
    try:
        if not os.path.exists(BASE_PATH):
            os.makedirs(BASE_PATH)
    except OSError:
        print("Creation of the directory %s failed" % BASE_PATH)
    else:
        print("Successfully created the directory %s" % BASE_PATH)

    grid_df = extract_features(train_df, prices_df, calendar_df, target=TARGET, nan_mask_d=1913+28)
    
    grid_df['item_id'] = grid_df['item_id'].astype('category')
    grid_df['dept_id'] = grid_df['dept_id'].astype('category')
    grid_df['cat_id'] = grid_df['cat_id'].astype('category')
#     grid_df['item_id'] = grid_df['item_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['dept_id'] = grid_df['dept_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['cat_id'] = grid_df['cat_id'].replace({'HOBBIES': 0, 'HOUSEHOLD': 1, 'FOODS': 2}).astype('category')
#     for col in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
#         grid_df[col] = grid_df[col].replace(
#             dict(zip(grid_df[col].unique(), np.arange(grid_df[col].unique().shape[0])))).astype('category')
    grid_df = reduce_mem_usage(grid_df)
    grid_df.to_pickle(TEMP_FEATURE_PKL)
    del grid_df

In [5]:
from module.catboost_baseline import *

In [6]:
useful_cols = dict(zip(STORES_IDS, [M5_FEATURES]*len(STORES_IDS)))

In [7]:
cat_params = {'iterations': 1400,
 'loss_function': 'Tweedie:variance_power=1.1',
 'eval_metric': 'Tweedie:variance_power=1.1',
 'subsample': 0.5,
 'learning_rate': 0.18,
 'random_strength': 0.5,
 'depth': 6,
 'verbose': 1,
 'random_seed': 42}

In [12]:
grid_df = pd.read_pickle(TEMP_FEATURE_PKL)
for col in ['item_id','dept_id','cat_id','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']:
    n_nan = grid_df[col].isna().sum()
    if n_nan > 0:
        grid_df[col] = grid_df[col].astype(str).fillna('NONE').astype('category')
        print(col, grid_df[col].dtype, grid_df[col].isna().sum())
grid_df.to_pickle(TEMP_FEATURE_PKL)
del grid_df

event_name_1 category 0
event_type_1 category 0
event_name_2 category 0
event_type_2 category 0


In [ ]:
history_df = train_evaluate_model(useful_cols, TARGET, BASE_PATH)#stores_ids=['CA_1']
print(history_df.rmse_trn.mean(), history_df.rmse_val.mean(), history_df.rmse_diff.mean())

Train CA_1
Fold: 0
Fold: 1
Fold: 2
Train CA_2
Fold: 0
Fold: 1
Fold: 2
Train CA_3
Fold: 0
Fold: 1
Fold: 2
Train CA_4
Fold: 0
Fold: 1
Fold: 2
Train TX_1
Fold: 0
Fold: 1
Fold: 2
Train TX_2
Fold: 0
Fold: 1
Fold: 2


In [ ]:
history_df.to_pickle(f'{BASE_PATH}/catboost_history_df.pkl')

In [13]:
history_df = train_evaluate_model(useful_cols, TARGET, BASE_PATH)#stores_ids=['CA_1']
print(history_df.rmse_trn.mean(), history_df.rmse_val.mean(), history_df.rmse_diff.mean())

Train CA_1
Fold: 0
Fold: 1
Fold: 2
Train CA_2
Fold: 0
Fold: 1
Fold: 2
Train CA_3
Fold: 0
Fold: 1
Fold: 2
Train CA_4
Fold: 0
Fold: 1
Fold: 2
Train TX_1
Fold: 0
Fold: 1
Fold: 2
Train TX_2
Fold: 0
Fold: 1
Fold: 2
Train TX_3
Fold: 0
Fold: 1
Fold: 2
Train WI_1
Fold: 0
Fold: 1
Fold: 2
Train WI_2
Fold: 0
Fold: 1
Fold: 2
Train WI_3
Fold: 0
Fold: 1
Fold: 2
1.815833035880111 2.329853734094095 0.5140206982139836


In [ ]:
final_all_preds = predict_test(useful_cols, TARGET, BASE_PATH)

FOLD0 Predict | Day:1
